In [1]:
#export
"""
For operations that feel like the termination
"""
from collections import defaultdict
from typing import Iterator, Any
from k1lib.cli.init import BaseCli, Table
import torch, numbers, numpy as np, k1lib; from k1lib import cli
__all__ = ["stdout", "file", "pretty", "display", "headOut", "intercept"]

In [2]:
#export
class stdout(BaseCli):
    """Prints out all lines. If not iterable, then print out the input raw.
Example::

    # prints out "0\\n1\\n2"
    range(3) | stdout()
    # same as above, but (maybe?) more familiar
    range(3) > stdout()"""
    def __ror__(self, it:Iterator[str]):
        try:
            it = iter(it)
            for line in it: print(line)
        except TypeError: print(it)

In [3]:
with k1lib.CaptureStdout() as out:
    range(3) > stdout()
assert out == ['0', '1', '2']

In [4]:
#export
class file(BaseCli):
    def __init__(self, fileName:str, text:bool=True):
        """Opens a new file for writing.
Example::

    # writes "0\\n1\\n2\\n" to file
    range(3) | file("test/f.txt")
    # same as above, but (maybe?) more familiar
    range(3) > file("text/f.txt")
    # returns ['0', '1', '2']
    cat("folder/f.txt") | deref()

    # writes bytes to file
    b'5643' | file("test/a.bin", False)
    # returns ['5643']
    cat("test/a.bin") | deref()

:param text: if True, accepts Iterator[str], and prints out each string on a
    new line. Else accepts bytes and write in 1 go."""
        super().__init__(); self.fileName = fileName; self.text = text
    def __ror__(self, it:Iterator[str]) -> None:
        super().__ror__(it)
        if self.text:
            with open(self.fileName, "w") as f:
                for line in it: f.write(f"{line}\n")
        else:
            with open(self.fileName, "wb") as f: f.write(it)

In [5]:
range(3) > file("test/f.txt")
assert cli.cat("test/f.txt") | cli.deref() == ['0', '1', '2']
b'5643' | file("test/a.bin", False)
assert cli.cat("test/a.bin") | cli.deref() == ['5643']

In [6]:
#export
class pretty(BaseCli):
    """Pretty prints a table. Not really used directly.
Example::

    # These 2 statements are pretty much the same
    [range(10), range(10)] | head(5) | pretty() > stdout()
    [range(10), range(10)] | display()"""
    def __ror__(self, it:Table[Any]) -> Iterator[str]:
        table = []; widths = defaultdict(lambda: 0)
        for row in it:
            _row = []
            for i, e in enumerate(row):
                e = f"{e}"; _row.append(e)
                widths[i] = max(len(e), widths[i])
            table.append(_row)
        for row in table:
            s = ""
            for w, e in zip(widths.values(), row):
                s += e.rstrip(" ").ljust(w+3)
            yield s
def display(lines:int=10):
    """Convenience method for displaying a table"""
    f = pretty() | stdout()
    if lines is None: return f
    else: return cli.head(lines) | f
def headOut(lines:int=10):
    """Convenience method for head() | stdout()"""
    if lines is None: return stdout()
    else: return cli.head(lines) | stdout()

In [7]:
with k1lib.CaptureStdout() as out1:
    [range(10), range(10)] | cli.head(5) | pretty() > stdout()
with k1lib.CaptureStdout() as out2:
    [range(10), range(10)] | display();
assert out1 == out2

In [8]:
#export
class intercept(BaseCli):
    def __init__(self, raiseError:bool=True):
        """Intercept flow at a particular point, analyze the object piped in, and
raises error to stop flow. Example::

    3 | intercept()

:param raiseError: whether to raise error when executed or not."""
        self.raiseError = raiseError
    def __ror__(self, s):
        print(type(s))
        if isinstance(s, (numbers.Number, str, bool)):
            print(k1lib.tab(f"{s}"))
        elif isinstance(s, (tuple, list)):
            print(k1lib.tab(f"Length: {len(s)}"))
            for e in s: print(k1lib.tab(f"- {type(e)}"))
        elif isinstance(s, (np.ndarray, torch.Tensor)):
            print(k1lib.tab(f"Shape: {s.shape}"))
            if s.numel() < 1000:
                print(k1lib.tab(f"{s}"))
        if self.raiseError: raise RuntimeError("intercepted")
        return s

In [9]:
try: 3 | intercept(); assert False
except: pass

<class 'int'>
    3


In [10]:
!../../export.py cli/output

Current dir: /home/kelvin/repos/labs/k1lib, ../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.9
Uninstalling k1lib-0.9:
  Successfully uninstalled k1lib-0.9
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing requirements to k1lib.egg-info/requires.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/fmt.py -> build/lib/k1lib
copying k1lib/selector.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib